In [ ]:
from google.colab import drive
drive.mount('/content/drive')

file_path = "/content/drive/MyDrive/normalized_dataset.csv"

Mounted at /content/drive


In [ ]:
import pandas as pd
import csv

data = []
with open(file_path, "r", encoding="utf-8") as file:
    reader = csv.reader(file)
    for row in reader:
        if len(row) >= 3:
            text = ",".join(row[1:-1]).strip()
            label = row[-1].strip()
            data.append((text, label))

df = pd.DataFrame(data, columns=["text", "label"])

In [ ]:
unique_labels = sorted(df["label"].unique())
label2id = {label: idx for idx, label in enumerate(unique_labels)}
id2label = {idx: label for label, idx in label2id.items()}
df["label"] = df["label"].map(label2id)

In [ ]:
from sklearn.model_selection import train_test_split

train_texts, val_texts, train_labels, val_labels = train_test_split(
    df["text"].tolist(), df["label"].tolist(), test_size=0.2, random_state=42
)

In [ ]:
from transformers import BertTokenizer
import torch
from torch.utils.data import Dataset, DataLoader

tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")

class ComplaintDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        encoding = self.tokenizer(
            self.texts[idx],
            truncation=True,
            padding='max_length',
            max_length=self.max_len,
            return_tensors="pt",
        )
        return {
            'input_ids': encoding['input_ids'].squeeze(),
            'attention_mask': encoding['attention_mask'].squeeze(),
            'labels': torch.tensor(self.labels[idx], dtype=torch.long)
        }

train_dataset = ComplaintDataset(train_texts, train_labels, tokenizer)
val_dataset = ComplaintDataset(val_texts, val_labels, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

In [ ]:
from transformers import BertForSequenceClassification, get_linear_schedule_with_warmup
from tqdm import tqdm
from torch.optim import AdamW

all_labels = train_labels + val_labels
label2id = {label: idx for idx, label in enumerate(sorted(set(all_labels)))}
label2id = {k: int(v) for k, v in label2id.items()}
id2label = {v: k for k, v in label2id.items()}

train_labels = [label2id[label] for label in train_labels]
val_labels = [label2id[label] for label in val_labels]

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BertForSequenceClassification.from_pretrained(
    "bert-base-multilingual-cased",
    num_labels=len(label2id),
    id2label=id2label,
    label2id=label2id
)
model.to(device)

optimizer = AdamW(model.parameters(), lr=2e-5)

model.train()
for batch in tqdm(train_loader, desc="Training"):
    optimizer.zero_grad()
    input_ids = batch["input_ids"].to(device)
    attention_mask = batch["attention_mask"].to(device)
    labels = batch["labels"].to(device)

    outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
    loss = outputs.loss
    loss.backward()
    optimizer.step()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Training: 100%|██████████| 500/500 [3:11:28<00:00, 22.98s/it]


In [ ]:
save_path = "/content/drive/MyDrive/multilingual_model"
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

('/content/drive/MyDrive/multilingual_model/tokenizer_config.json',
 '/content/drive/MyDrive/multilingual_model/special_tokens_map.json',
 '/content/drive/MyDrive/multilingual_model/vocab.txt',
 '/content/drive/MyDrive/multilingual_model/added_tokens.json')

In [ ]:
import json
with open(f"{save_path}/label_map.json", "w", encoding="utf-8") as f:
    json.dump({"label2id": label2id, "id2label": id2label}, f, ensure_ascii=False, indent=4)